In [2]:
import os
from PIL import Image, ImageOps

# --- USER INPUTS ---
IN_DIR  = r"./Nikon COOLPIX AW120"   # your input folder
OUT_DIR = r"./Cropped/Nikon COOLPIX AW120"    # choose where you want cropped images saved
H_M     = 2.0
F_MM    = 4.3
SENSOR_W_MM = 6.17
SENSOR_H_MM = 4.55
TARGET_GROUND_M = 1.0
USE_CONSERVATIVE = False

os.makedirs(OUT_DIR, exist_ok=True)

def compute_px_per_meter(img_w, img_h):
    ground_w_m = H_M * (SENSOR_W_MM / F_MM)
    ground_h_m = H_M * (SENSOR_H_MM / F_MM)
    px_per_m_x = img_w / ground_w_m
    px_per_m_y = img_h / ground_h_m
    if USE_CONSERVATIVE:
        return min(px_per_m_x, px_per_m_y)
    else:
        return 0.5 * (px_per_m_x + px_per_m_y)

def center_crop(im, crop_px):
    w, h = im.size
    cx, cy = w // 2, h // 2
    half = crop_px // 2
    return im.crop((cx - half, cy - half, cx + half, cy + half))

for fname in os.listdir(IN_DIR):
    if not fname.lower().endswith((".jpg", ".jpeg")):
        continue
    in_path = os.path.join(IN_DIR, fname)
    with Image.open(in_path) as im_raw:
        im = ImageOps.exif_transpose(im_raw)
        w, h = im.size

        px_per_m = compute_px_per_meter(w, h)
        crop_px = int(round(px_per_m * TARGET_GROUND_M))
        crop_px = min(crop_px, w, h)

        cropped = center_crop(im, crop_px)

        # filename: add "_c" before extension
        base, ext = os.path.splitext(fname)
        out_name = f"{base}_c{ext.lower()}"
        out_path = os.path.join(OUT_DIR, out_name)

        cropped.save(out_path, quality=95, subsampling=1, optimize=True)

print("Done cropping AW120 folder.")


Done cropping AW120 folder.
